In [ ]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Ok, now we'll have some fun and dig into _aggregations_ in SQL! Let's start with our first... The most basic, `COUNT`!

In [ ]:
%%sql
SELECT
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'

We can also count by state:

In [ ]:
%%sql
SELECT
    states,
    COUNT(*) as num_parks
FROM nps_public_data.parks p
WHERE designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 20;

But we note that states are defined as comma separate strings, so we'll need to split those strings to get a list of states for each park. We can do that with:

In [ ]:
%%sql
SELECT
    fullname,
    UNNEST(
        SPLIT(states, ',')::string[]
        ) as state
FROM nps_public_data.parks p
WHERE designation = 'National Park'
LIMIT 5

This _will_ produce multiple records for the same park (those in multiple states), but that's what we want! Now we can count the number of parks across state lines using an aggregation

In [ ]:
%%sql
WITH park_list AS (
    SELECT
        fullname,
        UNNEST(
            SPLIT(states, ',')::string[]
            ) as state
    FROM nps_public_data.parks p
    WHERE designation = 'National Park'
)
SELECT
    state,
    COUNT(*) as num_parks
FROM park_list
GROUP BY 1
ORDER BY 2 DESC, 1
LIMIT 10

Again, note a _pattern_: in order to count what we needed, we had to derive _one row per state per park_. Our original query gave us one row per park with mulitple states. So we _transformed our data_ as an intermediate step towards our final end.

This intermediate transformation is common in obtaining the format of data we need to perform our aggregation:

1. Inspect data and determine your aggregation, i.e. counting parks by state
2. Note the current _shape_ or format, i.e. at the state level
3. Note the _desired_ format, i.e. at the state-park level
4. Transform to get to the desired format, then aggregate

How do we find the campgrounds with the least and most sites using aggregations?

- Obtain the total number of campsites for each campground (that's a national park)
- Get the min / max numbers
- Filter the results by those numbers

In [ ]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), min_max_sites AS (
SELECT
    MIN(total_sites) as min_sites,
    MAX(total_sites) as max_sites
FROM park_campgrounds
WHERE total_sites > 0
)
SELECT
    campgroud_name,
    total_sites as num_sites,
    CASE total_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_campgrounds pc
INNER JOIN min_max_sites mms
    ON (pc.total_sites = mms.min_sites OR pc.total_sites = mms.max_sites)
ORDER BY num_sites, campgroud_name

What about the parks? We can follow similar logic

In [ ]:
%%sql
WITH park_campgrounds AS (
    SELECT
        c.name as campgroud_name,
        p.fullname as park_name,
        c.numberofsitesfirstcomefirstserve + c.numberofsitesreservable as total_sites,
    FROM nps_public_data.campgrounds c
    INNER JOIN nps_public_data.parks p
        ON c.parkcode = p.parkcode
        AND p.designation = 'National Park'
    GROUP BY 1,2,3
), park_sites AS (
    SELECT
        park_name,
        SUM(total_sites) as num_sites
    FROM park_campgrounds
    GROUP BY 1
    ORDER BY 2 DESC
), min_max_sites AS (
    SELECT
        MIN(num_sites) as min_sites,
        MAX(num_sites) as max_sites
    FROM park_sites ps
)
SELECT
    ps.*,
    CASE num_sites WHEN min_sites THEN 'least' ELSE 'most' END as sites_rank
FROM park_sites ps
INNER JOIN min_max_sites mms
    ON (num_sites = mms.min_sites or num_sites = mms.max_sites)
ORDER BY ps.num_sites DESC

Another important aggregation pattern is _using boolean logic within aggregations_ as filters! For example

- `COUNT DISTINCT`
- `COUNT CASE WHEN`

This lets us filter our count _without_ filtering the underlying data.

In [ ]:
%%sql
SELECT
    p.fullname as park_name,
    -- COUNT the number of campgrounds
    COUNT(DISTINCT c.name) as num_campgrounds,
    -- Get the average number of sites— what is this returning?
    AVG(numberofsitesreservable) as avg_sites_reservable,
    AVG(numberofsitesfirstcomefirstserve) as avg_sites_fcfs,
    AVG(numberofsitesreservable + numberofsitesfirstcomefirstserve) as avg_total_sites
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
-- Read more about aggregates here: https://duckdb.org/docs/sql/aggregates

In [ ]:
%%sql
SELECT
    p.name as park_name,
    COUNT(CASE WHEN c.numberofsitesreservable > 0 THEN 1 END) as num_reservable_campgrounds,
    COUNT(CASE WHEN c.numberofsitesfirstcomefirstserve > 0 THEN 1 END) as num_fcfs_campgrounds,
FROM nps_public_data.campgrounds c
INNER JOIN nps_public_data.parks p
    ON c.parkcode = p.parkcode
    AND p.designation = 'National Park'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5

There's lots more to aggregations, but they're pretty simple. The best way to get started will be to aggregate data in practice. Remember the basics:
- Aggregations collapse rows.
- Rows that _aren't_ being aggregated must be `GROUP`ed.
- `GROUP` statements appear at the end of a query.
- Duplicate rows may skew aggregates if not properly accounted for.